# Vector DB

In this notebook, we use a containerized version of Chroma DB. To set up, you will need the following:

1. Install [Docker Desktop](https://www.docker.com/products/docker-desktop/) by following the link and Download Docker Desktop for your operating system.
2. In a terminal window, navigate to the folder ./05_src/chromadb/. For example, on Windows, you would use `cd .\05_src\chromadb`.
3. Run the command `docker compose up -d`, which will start the Chroma DB server.

## Downloading Batch Results

In the previous notebook, we had created batch processes. We will start by consulting the status of our batch processes by identifying them throught their descriptions.

In [1]:
%load_ext dotenv
%dotenv ../../05_src/.secrets

In [2]:
batch_description = 'Pitchfork reviews content embeddings (jcalderon_20260212) 2026-02-12 14:22:54'

In [3]:
from openai import OpenAI
import os

# client = OpenAI(base_url='https://k7uffyg03f.execute-api.us-east-1.amazonaws.com/prod/openai/v1', 
#                 default_headers={"x-api-key": os.getenv('API_GATEWAY_KEY')})
client = OpenAI()

batch_processes = client.batches.list().to_dict()
batch_info= [
    {'batch_id': batch['id'],
     'description': batch['metadata']['description'],
    'status': batch['status'],
    'request_counts': batch['request_counts'],
    'output_file_id': batch['output_file_id'],
    'input_file_id': batch['input_file_id']}
            for batch in batch_processes['data'] if batch['metadata']['description'] == batch_description
    ]
batch_info

[{'batch_id': 'batch_698e289a55f881908160dc50245a838a',
  'description': 'Pitchfork reviews content embeddings (jcalderon_20260212) 2026-02-12 14:22:54',
  'status': 'completed',
  'request_counts': {'completed': 1000, 'failed': 0, 'total': 1000},
  'output_file_id': 'file-MCwdvNLhEq61ov9ZKmbqgn',
  'input_file_id': 'file-MjzJVjeK4guNZ7fWSiiAby'},
 {'batch_id': 'batch_698e289a1db881909344f9c3abd0f6d6',
  'description': 'Pitchfork reviews content embeddings (jcalderon_20260212) 2026-02-12 14:22:54',
  'status': 'completed',
  'request_counts': {'completed': 1000, 'failed': 0, 'total': 1000},
  'output_file_id': 'file-151NMxkjfoCfJknTFHrg3u',
  'input_file_id': 'file-1EiWYszPmEoUNm9AfTfEAA'},
 {'batch_id': 'batch_698e2899f0608190b8852c482cf342aa',
  'description': 'Pitchfork reviews content embeddings (jcalderon_20260212) 2026-02-12 14:22:54',
  'status': 'completed',
  'request_counts': {'completed': 1000, 'failed': 0, 'total': 1000},
  'output_file_id': 'file-1Ly9PDhvWQ27JoogSmeCuN',
 

In [4]:
batch_processes['data']

[{'id': 'batch_698e289a55f881908160dc50245a838a',
  'completion_window': '24h',
  'created_at': 1770924186,
  'endpoint': '/v1/embeddings',
  'input_file_id': 'file-MjzJVjeK4guNZ7fWSiiAby',
  'object': 'batch',
  'status': 'completed',
  'cancelled_at': None,
  'cancelling_at': None,
  'completed_at': 1770924342,
  'error_file_id': None,
  'errors': None,
  'expired_at': None,
  'expires_at': 1771010586,
  'failed_at': None,
  'finalizing_at': 1770924285,
  'in_progress_at': 1770924187,
  'metadata': {'description': 'Pitchfork reviews content embeddings (jcalderon_20260212) 2026-02-12 14:22:54',
   'timestamp': '2026-02-12 14:22:54'},
  'model': 'text-embedding-3-small',
  'output_file_id': 'file-MCwdvNLhEq61ov9ZKmbqgn',
  'request_counts': {'completed': 1000, 'failed': 0, 'total': 1000},
  'usage': {'input_tokens': 371224,
   'input_tokens_details': {'cached_tokens': 0},
   'output_tokens': 0,
   'output_tokens_details': {'reasoning_tokens': 0},
   'total_tokens': 371224}},
 {'id': 'b

When the status of the batches is complete, we can query the `output_file_id` where their results will be stored.

More generally, we will require the original text and the embeddings of that original text mapped through the `custom_id`.

In [5]:
batch_complete = [
    batch  for batch in batch_info if batch['status'] == 'completed'
]
batch_complete

[{'batch_id': 'batch_698e289a55f881908160dc50245a838a',
  'description': 'Pitchfork reviews content embeddings (jcalderon_20260212) 2026-02-12 14:22:54',
  'status': 'completed',
  'request_counts': {'completed': 1000, 'failed': 0, 'total': 1000},
  'output_file_id': 'file-MCwdvNLhEq61ov9ZKmbqgn',
  'input_file_id': 'file-MjzJVjeK4guNZ7fWSiiAby'},
 {'batch_id': 'batch_698e289a1db881909344f9c3abd0f6d6',
  'description': 'Pitchfork reviews content embeddings (jcalderon_20260212) 2026-02-12 14:22:54',
  'status': 'completed',
  'request_counts': {'completed': 1000, 'failed': 0, 'total': 1000},
  'output_file_id': 'file-151NMxkjfoCfJknTFHrg3u',
  'input_file_id': 'file-1EiWYszPmEoUNm9AfTfEAA'},
 {'batch_id': 'batch_698e2899f0608190b8852c482cf342aa',
  'description': 'Pitchfork reviews content embeddings (jcalderon_20260212) 2026-02-12 14:22:54',
  'status': 'completed',
  'request_counts': {'completed': 1000, 'failed': 0, 'total': 1000},
  'output_file_id': 'file-1Ly9PDhvWQ27JoogSmeCuN',
 

Before we download all results, examine the response of the file API:

In [6]:
response = client.files.content(batch_complete[0]['output_file_id'])
text_response = response.text
lines = text_response.split('\n')
print(lines[0])


{"id": "batch_req_698e28fdf7448190841d0b9fc9d5dfe8", "custom_id": "19643_2861_1807", "response": {"status_code": 200, "request_id": "51171700-6e23-42c1-a750-664697344570", "body": {"object": "list", "data": [{"object": "embedding", "index": 0, "embedding": [0.00030744667, 0.009419799, -0.00026936, 8.209572e-06, -0.02060168, -0.01085149, 0.009588665, 0.067194074, 0.0143829975, 0.0019346196, -0.010513758, -0.025359303, -0.023509117, 0.014104001, -0.034683656, 0.039646853, -0.008854464, -0.012914595, -0.0016574587, -0.028751312, 0.016284578, 0.009045356, -0.01355335, 0.027914321, -0.012995358, -0.015330117, -0.047165073, -0.0032451684, 0.049984403, 0.017429931, -0.020777889, -0.022525286, 0.030307816, 0.020058371, 0.002826674, -0.023641272, -0.014639968, 0.004368496, -0.044257637, 0.013090804, 0.030748338, -0.006453627, 0.01866339, -0.012356603, 0.02582919, 0.01303941, -0.012988016, 0.012261157, 0.015917478, 0.07594575, 0.004541033, 0.04463942, -0.018604653, 0.04690076, 0.023274172, -0.01

For our results database, we will need to map the original text to their embeddings. 

In [7]:
import json 

def get_text_and_embeddings(batch):
    embedding_lines =  get_content_from_file(batch, 'output_file_id')
    text_lines = get_content_from_file(batch, 'input_file_id')
    return embedding_lines, text_lines

def get_content_from_file(batch, key):
    file = client.files.content(batch[key])
    text = file.text
    lines = text.split('\n')
    content_lines = [json.loads(line) for line in lines if line.strip()]
    return content_lines


Notice that the response is also a .jsonl file. Therefore, we can process it line-by-line and use the `custom_id` to map to the original document chunk.

The function below:

- Creates a dictionary, `text_dict`, with keys given by each `custom_id` and value equal to the text.
- Iterate over all embedding items and use the dictionary defined above to map the embeddings to their input text.

In [8]:
def create_chroma_inputs(embedding_lines, text_lines):
    chroma_inputs = []
    text_dict = {item['custom_id']: item['body']['input'] for item in text_lines}
    for embed_item in embedding_lines:
        custom_id = embed_item['custom_id']
        text = text_dict.get(custom_id, "")
        chroma_input = {
            'id': embed_item['custom_id'],
            'embedding': embed_item['response']['body']['data'][0]['embedding'],
            'text': text
        }
        chroma_inputs.append(chroma_input)
    return chroma_inputs

A couple of functions to control the logic flow:

In [9]:
from tqdm import tqdm

def process_batch_for_chromadb(batch):
    embedding_lines, text_lines = get_text_and_embeddings(batch)
    chroma_inputs = create_chroma_inputs(embedding_lines, text_lines)
    return chroma_inputs

def process_batches_for_chromadb(batches):
    all_chroma_inputs = []
    for batch in tqdm(batches, desc="Processing batches"):
        chroma_inputs = process_batch_for_chromadb(batch)
        all_chroma_inputs.extend(chroma_inputs)
    return all_chroma_inputs

Now, we can create our input dictionaries.

In [10]:
chroma_inputs = process_batches_for_chromadb(batch_complete)

Processing batches: 100%|██████████| 20/20 [00:55<00:00,  2.77s/it]


In [11]:
chroma_inputs[1]

{'id': '19643_2861_3609',
 'embedding': [-0.008924569,
  -0.029277397,
  -0.02974355,
  0.03888616,
  0.0344953,
  0.026931597,
  -0.011947043,
  0.054314308,
  0.0075223516,
  0.0054547386,
  -0.00020629137,
  -0.019307744,
  -0.022239996,
  0.0013232722,
  -0.040961288,
  0.017864175,
  -0.02888643,
  -0.037953854,
  0.00667651,
  -0.008022338,
  0.002039418,
  0.0021427989,
  -0.039126754,
  0.026826337,
  -0.020119753,
  -0.008225339,
  0.0030525483,
  0.010165136,
  0.045412295,
  0.017849138,
  -0.016044676,
  -0.034134407,
  0.03491634,
  0.0046803234,
  -0.006755455,
  -0.029442806,
  -0.043698058,
  0.031337492,
  -0.009706503,
  -0.0008575894,
  0.04460029,
  -0.029908959,
  0.02467602,
  -0.013398131,
  0.004368302,
  -0.014661254,
  -0.05659996,
  -0.0030619467,
  0.04357776,
  0.0639381,
  -0.014112397,
  0.050976053,
  -0.038104225,
  0.03669073,
  0.0146236615,
  -0.009947097,
  -0.014127434,
  -0.01201471,
  0.0014454493,
  0.00072695385,
  0.011413223,
  0.023036966,
 

In [13]:

with open('../../05_src/documents/chroma_inputs.jsonl', 'w') as f:
    for ci in tqdm(chroma_inputs):
        json_str = json.dumps(ci)
        f.write(json_str + '\n')

100%|██████████| 19346/19346 [00:16<00:00, 1155.26it/s]


# Load Embeddings to Chroma

In [14]:
chroma_inputs = []
with open('../../05_src/documents/chroma_inputs.jsonl', 'r') as f:
    for line in f:
        chroma_inputs.append(json.loads(line.strip()))


In [15]:

import chromadb
from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction
import os


def setup_collection(chroma_url:str="http://localhost:8000",
                     collection_name: str = "pitchfork_reviews"):
    chroma_client = chromadb.HttpClient(host=chroma_url)
    collections = chroma_client.list_collections()
    if collection_name in [col.name for col in collections]:
        chroma_client.delete_collection(name=collection_name)

    collection = chroma_client.create_collection(
        name=collection_name,
        embedding_function=OpenAIEmbeddingFunction(
            api_key = os.getenv("OPENAI_API_KEY"),
            model_name="text-embedding-3-small")
        )
    return collection

def load_embeddings_to_db(chroma_inputs:list[dict], 
                          collection_name:str,
                          chroma_url:str="http://localhost:8000",
                          batch_size:int= 1000
                          ):

    
    collection = setup_collection(chroma_url=chroma_url, collection_name=collection_name)

    for i in tqdm(range(0, len(chroma_inputs), batch_size)):
        batch = chroma_inputs[i:i + batch_size]
        collection.add(
            documents=[item['text'] for item in batch],
            embeddings=[item['embedding'] for item in batch],
            ids=[item['id'] for item in batch]
        )


In [16]:
vector_db_client_url:str="http://localhost:8000"
load_embeddings_to_db(chroma_inputs=chroma_inputs,
                      collection_name="pitchfork_reviews",
                      chroma_url=vector_db_client_url, 
                      batch_size=1000)

100%|██████████| 20/20 [00:54<00:00,  2.70s/it]


# Additional Details

We will use a simple database to store additional details about the reviews. In this case, we load the jsonl files, and use pandas to create a few tables in a sql database. The connection string to the database is included in the .secrets file.

In [17]:
import json

def load_jsonl(file:str):
    data = []
    with open(file, 'r', encoding='utf-8') as f:
        for line in f:
            if line.strip():
                data.append(json.loads(line))
    return data

In [18]:
import pandas as pd
import sqlalchemy as sa
import os

doc_folder = "../../05_src/documents/"
tables = ["artists", "reviews", "labels", "genres"]

def upload_tables_to_sql(tables:list[str], doc_folder:str):
    engine = sa.create_engine(os.getenv("SQL_URL"))
    for table_name in tables:
        file_path = os.path.join(doc_folder, f"pitchfork_{table_name}.jsonl")
        data = load_jsonl(file_path)
        df = pd.DataFrame(data)
        with engine.connect() as conn:
            df.to_sql(table_name, conn, if_exists='replace', index=False)
        print(f"Loaded {df.shape} records from {file_path}")

upload_tables_to_sql(tables=tables, doc_folder=doc_folder)

Loaded (18831, 2) records from ../../05_src/documents/pitchfork_artists.jsonl
Loaded (18393, 13) records from ../../05_src/documents/pitchfork_reviews.jsonl
Loaded (20190, 2) records from ../../05_src/documents/pitchfork_labels.jsonl
Loaded (22680, 2) records from ../../05_src/documents/pitchfork_genres.jsonl


In [19]:
def additional_details(review_id:str):
    import sqlalchemy as sa
    import pandas as pd
    import os

    engine = sa.create_engine(os.getenv("SQL_URL"))
    query = f"""
    SELECT r.reviewid,
		r.title,
		r.artist,
		r.score,
		g.genre
    FROM reviews AS r
    LEFT JOIN genres as g
	    ON r.reviewid = g.reviewid
    WHERE r.reviewid = '{review_id}'
    """
    with engine.connect() as conn:
        result = pd.read_sql(query, conn)
    if not result.empty:
        row = result.iloc[0]
        details = {
            "reviewid": row['reviewid'],
            "album": row['title'],
            "score": row['score'],
            "artist": row['artist']
        }
        return details
    else:
        return {}
    
def get_reviewid_from_custom_id(custom_id:str):
    return custom_id.split('_')[0]

# Prompt Generator

Here we create a prompt with the context gathered through the different data operations.

In [20]:
chroma = chromadb.HttpClient(host=vector_db_client_url)
collection = chroma.get_collection(name="pitchfork_reviews", 
                                   embedding_function=OpenAIEmbeddingFunction(
                                       api_key = os.getenv("OPENAI_API_KEY"),
                                       model_name="text-embedding-3-small")
                                   )


In [21]:
collection.query(
    query_texts=["A great album with stunning vocals and production."],
    n_results=3
)

{'ids': [['398_15632_1801', '4428_16669_1802', '6881_16771_3609']],
 'distances': [[1.0196459, 1.0360501, 1.036174]],
 'embeddings': None,
 'metadatas': [[None, None, None]],
 'documents': [['vocals take on an even more tranquil quality than previous offerings, just begging to grace     a pair of headphones.  In fact, the subtle variations in vocal renderings between the three songs are the     only true variants within the album, which follows logically given no-one has any misconceptions about     where Azure Ray\'s bread is buttered.          "The Love of Two", the only non-album track, is the most atmospheric, as well as most memorable.  An     incessant vinyl scratch murmurs beneath a subtle vocal tonality reminiscent of Mazzy Star, which, distant     and haunted, provides an excellent equipoise to the visceral production.  A "Bleed Version" of the     album-inclusive "We Are Mice" rounds out the disc with sparse piano backing and choral vocals that I     swear could only be descr

In [22]:
def get_context_data(query:str, collection:chromadb.api.models.Collection, top_n:int):
    results = collection.query(
        query_texts=[query],
        n_results=top_n
    )
    context_data = []
    for idx, custom_id in enumerate(results['ids'][0]):
        review_id = get_reviewid_from_custom_id(custom_id)
        details = additional_details(review_id)
        details['text'] = results['documents'][0][idx]
        context_data.append(details)
    return context_data

def generate_prompt(query:str, collection:chromadb.api.models.Collection, top_n:int):
    context_data = get_context_data(query, collection, top_n)
    prompt = f"Given a query, provide a detailed response using the context from relevant Pitchfork reviews. The context will contain references to {top_n} album reviews.\n\n"
    prompt += f"The score is numeric and its scale is from 0 to 10, with 10 being the highest rating. Any album with a score greater than 8.0 is considered a must-listen; album with a score greater than 6.5 is good.\n\n"
    prompt += f"<query>{query}</query>\n\n"
    prompt += "<context>\n"
    for k, context in enumerate(context_data):
        prompt += f"<album {k}>\n"
        prompt += f"- Album Title: {context.get('album', 'N/A')}\n" 
        prompt += f"- Album Artist: {context.get('artist', 'N/A')}\n"
        prompt += f"- Album Score: {context.get('score', 'N/A')}\n"
        prompt += f"- Review Quote: {context.get('text', 'N/A')}\n"
        prompt += f"</album {k}>\n\n"
    prompt += "</context>\n\n"
    prompt += "\nBased on the context and nothing else, provide a detailed response to the query."
    return prompt

def generate_response(query:str, collection:chromadb.api.models.Collection, top_n:int=1):
    prompt = generate_prompt(query, collection, top_n)
    print("Generated Prompt:\n", prompt)
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are a helpful assistant that provides information based on Pitchfork reviews."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=500,
        temperature=0.7
    )
    return response.choices[0].message.content

# Query

We can now use chroma's similarity function to query the database. Notice that the query itself needs to be converted to embeddings, so we must provide an `embedding_function`. In this case, we use `OpenAIEmbeddingFunction()` to get compatible embeddings using model `text-embedding-3-small`.

In [23]:
response = generate_response("What are some highly rated albums by emerging indie artists?", collection, 3)

Generated Prompt:
 Given a query, provide a detailed response using the context from relevant Pitchfork reviews. The context will contain references to 3 album reviews.

The score is numeric and its scale is from 0 to 10, with 10 being the highest rating. Any album with a score greater than 8.0 is considered a must-listen; album with a score greater than 6.5 is good.

<query>What are some highly rated albums by emerging indie artists?</query>

<context>
<album 0>
- Album Title: conductor
- Album Artist: the comas
- Album Score: 8.0
- Review Quote: At the heart of the indie mindset is a profound disconnect between perception and actuality.  The ideal:     D.I.Y. ethics, an emphasis on art above marketability, and a purity of purpose that is resistant to fads and     monetary influence.  The reality: indie bands are receiving six-digit recording advances from major media     conglomerates, and are just as ripe for demographic targeting, strategic media placement, and lifestyle     brandi

In [24]:
print(response)

If you’re looking for highly rated albums by emerging indie artists, there are a couple that stand out based on recent Pitchfork reviews.

1. **Espers - *Espers*** (Score: 8.4)
   This self-titled album from the band Espers is a notable entry in the indie scene, particularly for its homage to the folk music of the late 60s and early 70s. The review highlights a resurgence of interest in folk aesthetics within the indie community, with Espers contributing a lush blend of psych-infused melancholia and beautifully baroque melodies. Their sound reflects a deep connection to the folk revival, showcasing a refreshing take that resonates with both old and new listeners. This album is highly recommended for anyone interested in the current wave of folk-inspired indie music.

2. **The Comas - *Conductor*** (Score: 8.0)
   The Comas offer a unique perspective on the indie music landscape with *Conductor*, navigating the often complex relationship between indie and mainstream music. The review di

**Note**: Try changing the top_n parameter to 1 and re-run the query. 